In [ ]:
# ✅ Step 1: Install Required Packages (Run only once in Colab)
!pip install sentence-transformers faiss-cpu


In [ ]:
# ✅ Step 2: Import Libraries
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:
# ✅ Step 3: Load Dataset
df = pd.read_csv("cleaned_carbonated_drinks.csv")
df.fillna("", inplace=True)

In [ ]:
# ✅ Step 4: Hybrid Text Embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

def create_hybrid_text(row):
    return f"{row['FlavorProfile']} {row['UseCase']} {row['HealthTags']} {row['Type']}"

df['hybrid_text'] = df.apply(create_hybrid_text, axis=1)
drink_embeddings = model.encode(df['hybrid_text'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
# ✅ Step 5: Build FAISS Index
dimension = drink_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(drink_embeddings))


In [ ]:
# ✅ Step 6: User Input Interface
def get_user_input():
    print("🍹 Welcome to DrinkBot! Type 'exit' to quit.\n")
    username = input("👤 Enter your username: ")
    if username.lower() == 'exit':
        return None
    flavor = input("🔀 What flavor are you in the mood for? ").strip()
    if flavor.lower() == 'exit':
        return None
    use_case = input("🔀 What's the occasion or use case? ").strip()
    tags = input("🔀 Any health preference? (e.g., Diabetic-safe, Low-calorie): ").strip()
    drink_type = input("🔀 Preferred drink type? (Soda, Sparkling Water, etc.) or press Enter to skip: ").strip()
    return {
        'flavor': flavor,
        'use_case': use_case,
        'tags': tags,
        'type': drink_type
    }


In [ ]:
# ✅ Step 7: Generate Hybrid Query Embedding
def get_query_embedding(user_input):
    query = f"{user_input['flavor']} {user_input['use_case']} {user_input['tags']} {user_input['type']}"
    return model.encode([query])[0]

In [ ]:
# ✅ Step 8: Get Top-K Recommendations
def get_top_k_recommendations(query_embedding, k=10):
    D, I = index.search(np.array([query_embedding]), k)
    return df.iloc[I[0]]

In [ ]:
# ✅ Step 9: Evaluation Functions
def get_relevant_items(user_input, df):
    relevant = []
    for _, row in df.iterrows():
        if user_input['flavor'].lower() in row['FlavorProfile'].lower():
            relevant.append(row['ProductName'])
    return relevant

def evaluate_model(user_input, recommendations, k):
    relevant = get_relevant_items(user_input, df)
    retrieved = recommendations['ProductName'].tolist()
    retrieved_k = retrieved[:k]

    y_true = [1 if item in relevant else 0 for item in retrieved_k]
    y_pred = [1] * len(retrieved_k)

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    accuracy = sum([1 for item in retrieved_k if item in relevant]) / k * 100

    return accuracy, precision, recall, f1

In [ ]:
# ✅ Step 10: Main Bot Loop
while True:
    user_input = get_user_input()
    if user_input is None:
        break

    query_embedding = get_query_embedding(user_input)
    top_k_df = get_top_k_recommendations(query_embedding, k=100)

    print("\n🤖 DrinkBot:\nHere are your top 10 personalized drinks:\n")
    for idx, row in top_k_df.head(10).iterrows():
        print(f"{idx+1}. Name: {row['ProductName']}")
        print(f"   Flavor: {row['FlavorProfile']}")
        print(f"   Use Case: {row['UseCase']}")
        print(f"   Tags: {row['HealthTags']}")
        print(f"   Calories: {row['Calories']}")
        print(f"   Caffeine: {row['Caffeine_mg']} mg")
        print(f"   Type: {row['Type']}\n")

    print("📊 Evaluation Metrics:")
    for k in [5, 10, 30, 50, 100]:
        acc, precision, recall, f1 = evaluate_model(user_input, top_k_df, k)
        print(f"\n@{k} Recommendations:")
        print(f" - Accuracy: {acc:.2f}%")
        print(f" - Precision: {precision:.2f}")
        print(f" - Recall: {recall:.2f}")
        print(f" - F1 Score: {f1:.2f}")

🍹 Welcome to DrinkBot! Type 'exit' to quit.

👤 Enter your username: abc
🔀 What flavor are you in the mood for? mint
🔀 What's the occasion or use case? party
🔀 Any health preference? (e.g., Diabetic-safe, Low-calorie): Diabetic-afe
🔀 Preferred drink type? (Soda, Sparkling Water, etc.) or press Enter to skip: 

🤖 DrinkBot:
Here are your top 10 personalized drinks:

633. Name: Limca Mint Blast
   Flavor: Mint
   Use Case: Party
   Tags: Sugar-free
   Calories: 160
   Caffeine: 0 mg
   Type: Tonic

340. Name: Pepsi Mint Blast
   Flavor: Mint
   Use Case: Party
   Tags: Sugar-free
   Calories: 20
   Caffeine: 160 mg
   Type: Soda

590. Name: Appy Fizz Mint Blast
   Flavor: Mint
   Use Case: Party
   Tags: Sugar-free
   Calories: 0
   Caffeine: 40 mg
   Type: Soda

400. Name: Limca Mint Blast
   Flavor: Mint
   Use Case: Party
   Tags: Sugar-free, Zero calorie, High caffeine
   Calories: 0
   Caffeine: 160 mg
   Type: Sparkling Juice

366. Name: Red Bull Mint Blast
   Flavor: Mint
   Use Cas